### Installs and Imports

In [1]:
!pip install -q stable-baselines3[extra]
!pip install -q sb3-contrib
!pip install -q optuna

     |████████████████████████████████| 177 kB 31.9 MB/s 
     |████████████████████████████████| 1.5 MB 81.7 MB/s 
     |████████████████████████████████| 1.1 MB 63.8 MB/s 
     |████████████████████████████████| 1.6 MB 66.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 308 kB 40.3 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 209 kB 75.9 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 147 kB 50.5 MB/s 
     |████████████████████████████████| 112 kB 91.8 MB/s 


In [2]:
import torch
import torch.nn as nn

import gym
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [33]:
N_TRIALS = 20 # maximum number of trials
N_JOBS = 1 # number of jobs to run in parallel
N_STARTUP_TRIALS = 5 # do N_STARTUP_TRIALS random sampling
N_EVALUATIONS = 2 # number of evaluations to run during training
N_TIMESTEPS = int(2e4) # training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)

N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60*15)  # 15 minutes

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID
}

### Search Space

In [34]:
from typing import Any, Dict

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
  """
  Sampler for A2C parameters

  :param trial: Optuna trial object
  :return: The sampled hyperparemters dictionary for a given trial
  """
  gamma = 1 - trial.suggest_float("gamma_", 1e-4, 0.1, log=True)
  lr = trial.suggest_float("lr", 1e-5, 1, log=True)
  n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
  max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5, log=True)

  net_arch = trial.suggest_categorical("arch", ["small", "tiny"])
  activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

  # Display true values
  trial.set_user_attr("gamma", gamma)
  trial.set_user_attr("n_steps", n_steps)

  # Set network architecture and activation function from categorical suggestion
  net_arch = [
      {"pi": [64], "vf": [64]}
      if net_arch == "tiny"
      else {"pi": [64, 64], "vf": [64, 64]}]
  activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]


  return {"gamma": gamma,
          "learning_rate": lr,
          "n_steps": n_steps,
          "max_grad_norm": max_grad_norm,
          "policy_kwargs": {"net_arch": net_arch,
                            "activation_fn": activation_fn}
          }

### Evaluation Callbak Class

In [35]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
  """
  Callback used for evaluating and reporting a trial

  :param eval_env: Evaluation environment
  :param trial: Optuna trial object
  :param: n_eval_episodes: Number of episodes used to evalaute policy
  :param: eval_freq: Number of successive steps after which policy is evaluated 
      during training
  :param: deterministic: whether the evaluation should use a deterministic or 
      stochastic policy
  :param: verbose: verbosity
  """
  def __init__(
      self,
      eval_env: gym.Env,
      trial: optuna.Trial,
      n_eval_episodes: int = 5,
      eval_freq: int = 10_000,
      deterministic: bool = True,
      verbose: int = 0):
    
    super().__init__(
        eval_env=eval_env,
        n_eval_episodes=n_eval_episodes,
        eval_freq=eval_freq,
        deterministic=deterministic,
        verbose=verbose)
    self.trial = trial
    self.is_pruned = False
    self.eval_idx = 0

  def _on_step(self) -> bool:
    if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
      # Evalaute the policy. Done in the parent class
      super()._on_step()
      self.eval_idx += 1

      # Send report to Optuna
      self.trial.report(self.last_mean_reward, self.eval_idx)

      # Prune trial if needed
      if self.trial.should_prune():
        self.is_pruned = True
        return False
      return True       

### Objective Function

In [36]:
def objective(trial: optuna.Trial) -> float:
  """
  Objective function used by Optuna to evaluate one configuration (trial, i.e. 
  a set of parameters)

  Given a trial object, it will sample one set of hyperparameters, evaluate it, 
  and report the result (mean episodic reward)

  :param trial: Optuna trial object
  :return: Mean episodic reward after training 
  """

  # Initialize with default hyperparameters
  kwargs = DEFAULT_HYPERPARAMS.copy()

  # Update hyperparameters
  kwargs.update(sample_a2c_params(trial))

  # Create the model
  model = A2C(**kwargs)

  # Create the evaluation env
  eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS)

  # Create EvalCallback object
  # TrailEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
  eval_callback = TrialEvalCallback(eval_envs, trial, N_EVAL_EPISODES, EVAL_FREQ, True, 1)

  # Train the model
  nan_encountered = False

  try:
    model.learn(N_TIMESTEPS, callback=eval_callback)
  except AssertionError as e:
    # Sometimes, random parameters can generate NaN
    print(e)
    nan_encountered = True
  finally:
    model.env.close()
    eval_envs.close()

  # Tell the optimizer that the trial failed
  if nan_encountered: return float("nan")

  if eval_callback.is_pruned: raise optuna.exceptions.TrialPruned()
  return eval_callback.last_mean_reward

### The Optimization Loop

In [37]:
# Set PyTorch number of threads to 1 for a faster training
torch.set_num_threads(1)

# Select the sampler. It can be RandomSampler, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)

# Select the pruner. Do not prune before 1/3 of the maximum budget is used
# Do not prune before (N_EVALUATIONS // 3) evaluations
pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps = N_EVALUATIONS // 3)

# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
  study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
  pass

[I 2022-08-26 15:15:50,998] A new study created in memory with name: no-name-e9239167-b171-40bc-b007-d2a8353d2311


Eval num_timesteps=10000, episode_reward=9.20 +/- 0.60
Episode length: 9.20 +/- 0.60
New best mean reward!


[I 2022-08-26 15:16:04,354] Trial 0 finished with value: 9.2 and parameters: {'gamma_': 0.08713821351614517, 'lr': 0.18721093064885544, 'exponent_n_steps': 3, 'max_grad_norm': 1.5370488337131525, 'arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: 9.2.


Eval num_timesteps=20000, episode_reward=9.20 +/- 0.98
Episode length: 9.20 +/- 0.98
Eval num_timesteps=10000, episode_reward=9.40 +/- 0.66
Episode length: 9.40 +/- 0.66
New best mean reward!


[I 2022-08-26 15:16:16,392] Trial 1 finished with value: 9.6 and parameters: {'gamma_': 0.00854047343750428, 'lr': 0.594188308348942, 'exponent_n_steps': 3, 'max_grad_norm': 0.3123929494111596, 'arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 1 with value: 9.6.


...

Eval num_timesteps=10000, episode_reward=100.60 +/- 37.64
Episode length: 100.60 +/- 37.64
New best mean reward!
Eval num_timesteps=10000, episode_reward=328.10 +/- 82.27
Episode length: 328.10 +/- 82.27
New best mean reward!


[I 2022-08-26 15:18:27,877] Trial 18 finished with value: 500.0 and parameters: {'gamma_': 0.0009394398266475002, 'lr': 0.007549043894468425, 'exponent_n_steps': 9, 'max_grad_norm': 0.46607452438223707, 'arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 8 with value: 500.0.


Eval num_timesteps=20000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!


[I 2022-08-26 15:18:31,688] Trial 19 pruned. 


Eval num_timesteps=10000, episode_reward=66.50 +/- 21.86
Episode length: 66.50 +/- 21.86
New best mean reward!


In [38]:
# Print Results
print("Number of finished trials", len(study.trials))

trial = study.best_trial
print("Best trial:")
print(f"  Mean Reward: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
  print(f"    {key}: {value}")
print("  User Attributes:")
for key, value in trial.user_attrs.items():
  print(f"    {key}: {value}")


# Write Report
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

Number of finished trials 20
Best trial:
  Mean Reward: 500.0
  Params: 
    gamma_: 0.0009943486624725889
    lr: 0.010135413604588724
    exponent_n_steps: 9
    max_grad_norm: 0.8375117706725724
    arch: tiny
    activation_fn: relu
  User Attributes:
    gamma: 0.9990056513375274
    n_steps: 512
